In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('wisataindonesia.csv')
data.head()

,attraction_id,nama,category,total_review,total_rating,img,longitude,latitude,link_detail,kota,provinsi,adult_price,child_price
0,1,Geopark Ciletuh,Nature,61,4.5 of 5 bubbles,https://dynamic-media-cdn.tripadvisor.com/medi...,0.000000,0.000000,https://www.tripadvisor.com/Attraction_Review-...,None,None,110000,85000
1,2,Ujung Genteng Beach,Beaches,192,4.0 of 5 bubbles,https://dynamic-media-cdn.tripadvisor.com/medi...,106.401973,-7.361441,https://www.tripadvisor.com/Attraction_Review-...,NaN,West Java,80000,55000
2,3,Jembatan Gantung Situ Gunung,Bridges,48,4.5 of 5 bubbles,https://dynamic-media-cdn.tripadvisor.com/medi...,106.928887,-6.831050,https://www.tripadvisor.com/Attraction_Review-...,Kabupaten Sukabumi,Jawa Barat,70000,45000
3,4,Pelabuhan Ratu,Neighborhoods,81,4.0 of 5 bubbles,https://dynamic-media-cdn.tripadvisor.com/medi...,106.552363,-6.988996,https://www.tripadvisor.com/Attraction_Review-...,Sukabumi Regency,West Java,120000,95000
4,5,Curug Awang,Waterfalls,9,4.5 of 5 bubbles,https://dynamic-media-cdn.tripadvisor.com/medi...,106.510475,-7.259570,https://www.tripadvisor.com/Attraction_Review-...,Sukabumi Regency,West Java,90000,65000


In [30]:
#YANG pake cosine similarity
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.feature_extraction.text import TfidfVectorizer
from datetime import datetime
import pickle
from sklearn.metrics.pairwise import cosine_similarity

# Menghapus sesi TensorFlow sebelumnya
tf.keras.backend.clear_session()

# Memuat dataset
data = pd.read_csv('wisataindonesia.csv')

# Pra-pemrosesan data
data['kota'] = data['kota'].fillna('Unknown')
data['provinsi'] = data['provinsi'].fillna('')

# Melakukan vektorisasi TF-IDF pada fitur "kota" dan "provinsi"
tfidf_kota = TfidfVectorizer()
tfidf_provinsi = TfidfVectorizer()

tfidf_matrix_kota = tfidf_kota.fit_transform(data['kota'])
tfidf_matrix_provinsi = tfidf_provinsi.fit_transform(data['provinsi'])

# Menggabungkan matriks TF-IDF
tfidf_matrix = np.concatenate((tfidf_matrix_kota.toarray(), tfidf_matrix_provinsi.toarray()), axis=1)
with open('tfidf_matrix.pickle', 'wb') as file:
    pickle.dump(tfidf_matrix, file)

# # Membangun model rekomendasi
input_shape = tfidf_matrix.shape[1]
input_layer = Input(shape=(input_shape,))
hidden_layer = Dense(128, activation='relu')(input_layer)
output_layer = Dense(input_shape, activation='linear')(hidden_layer)

model = Model(inputs=input_layer, outputs=output_layer)

# Membangun model rekomendasi
# input_shape = tfidf_matrix.shape[1]
# model = Sequential()
# model.add(Dense(128, activation='relu', input_shape=(input_shape,)))
# model.add(Dense(input_shape, activation='linear'))
model.compile(loss='cosine_similarity', optimizer=Adam(learning_rate=0.001))

# Melatih model
model.fit(tfidf_matrix, tfidf_matrix, epochs=10, batch_size=32)

# Pickle model
# with open('recommendation_model.pkl', 'wb') as f:
#     pickle.dump(model, f)

# Mendapatkan embedding item
model.save('recommendation_model.h5')

# Mendapatkan indeks item berdasarkan input kota
def get_item_index_by_kota(kota, data):
    index = data[data['kota'] == kota].index
    return index[0] if len(index) > 0 else None

from math import radians, sin, cos, sqrt, atan2

# Menghitung jarak antara dua titik berdasarkan lintang dan bujur formula haversine
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Jari-jari bumi dalam kilometer

    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    a = sin(dlat/2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon/2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    distance = R * c
    return distance

import random

# Merekomendasikan item berdasarkan kota dan durasi liburan yang diberikan
def recommend_items(kota, durasi, items=data[['attraction_id', 'nama', 'kota', 'provinsi','longitude','latitude','img','total_rating']], k=5):
    indeks_item = get_item_index_by_kota(kota, data)
    if indeks_item is None:
        return pd.DataFrame()  # Mengembalikan dataframe kosong jika kota tidak ditemukan
    item_embedding = tfidf_matrix[indeks_item]
    item_embedding = np.reshape(item_embedding, (1, -1))
    
    item_embedding_pred = model.predict(item_embedding)
    similarity_scores_pred = cosine_similarity(tfidf_matrix, item_embedding_pred)
    similarity_scores = similarity_scores_pred.flatten()

    # Tambahkan faktor acak pada similarity_scores
    random_factor = np.random.normal(0, 0.01, size=similarity_scores.shape)
    similarity_scores += random_factor

    indeks_terurut = np.argsort(similarity_scores)[::-1][:k]
    item_terrekomendasikan = items.iloc[indeks_terurut].copy()  # Add .copy() here

    # Pengurutan acak item_terrekomendasikan
    item_terrekomendasikan = item_terrekomendasikan.sample(frac=1, random_state=42)

    # Membagi item terrekomendasikan secara merata berdasarkan durasi liburan
    jumlah_terrekomendasikan = len(item_terrekomendasikan)
    jumlah_hari = durasi
    item_per_hari = jumlah_terrekomendasikan // jumlah_hari
    sisa = jumlah_terrekomendasikan % jumlah_hari

    alokasi_hari = [item_per_hari] * jumlah_hari
    for i in range(sisa):
        alokasi_hari[i] += 1

    item_terrekomendasikan['hari'] = np.repeat(range(1, jumlah_hari + 1), alokasi_hari)[:jumlah_terrekomendasikan]
    item_terrekomendasikan = item_terrekomendasikan.sort_values('hari')

    # Batasi jumlah atraksi per hari menjadi 3
    item_terrekomendasikan = item_terrekomendasikan.groupby('hari').head(3)

    # Menghitung jarak antara atraksi dan kota yang diberikan
    lintang_kota = items.loc[indeks_item, 'latitude']
    bujur_kota = items.loc[indeks_item, 'longitude']
    item_terrekomendasikan['jarak'] = item_terrekomendasikan.apply(
        lambda row: calculate_distance(row['latitude'], row['longitude'], lintang_kota, bujur_kota),
        axis=1
    )
    item_terrekomendasikan = item_terrekomendasikan.sort_values('jarak')

    return item_terrekomendasikan



kota_input = 'Kabupaten Bantul'  # Ganti dengan input kota yang diinginkan
durasi_input = 2# Ganti dengan durasi liburan yang diinginkan

item_terrekomendasikan = recommend_items(kota_input, durasi_input,items=data[['attraction_id', 'nama', 'kota', 'provinsi','longitude','latitude','img','total_rating','category','adult_price','child_price']], k=20)
if not item_terrekomendasikan.empty:
#     item_terrekomendasikan.to_csv('recommended_items.csv', index=False)
    print('Item yang direkomendasikan disimpan ke recommended_items.csv')
else:
    print('Tidak ada item yang ditemukan untuk kota yang diberikan.')


Epoch 1/10
38/38 [==============================] - 0s 2ms/step - loss: -0.2832
Epoch 2/10
38/38 [==============================] - 0s 3ms/step - loss: -0.6444
Epoch 3/10
38/38 [==============================] - 0s 2ms/step - loss: -0.7982
Epoch 4/10
38/38 [==============================] - 0s 2ms/step - loss: -0.8639
Epoch 5/10
38/38 [==============================] - 0s 2ms/step - loss: -0.8968
Epoch 6/10
38/38 [==============================] - 0s 2ms/step - loss: -0.9166
Epoch 7/10
38/38 [==============================] - 0s 3ms/step - loss: -0.9299
Epoch 8/10
38/38 [==============================] - 0s 3ms/step - loss: -0.9391
Epoch 9/10
38/38 [==============================] - 0s 2ms/step - loss: -0.9463
Epoch 10/10
1/1 [==============================] - 0s 69ms/step
Item yang direkomendasikan disimpan ke recommended_items.csv


In [31]:
item_terrekomendasikan

,attraction_id,nama,kota,provinsi,longitude,latitude,img,total_rating,category,adult_price,child_price,hari,jarak
886,887,Ganjuran Church,Kabupaten Bantul,Daerah Istimewa Yogyakarta,110.319129,-7.926458,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5 of 5 bubbles,Churches & Cathedrals,100000,75000,2,0.000000
904,905,Selarong Cave,Kabupaten Bantul,Daerah Istimewa Yogyakarta,110.314867,-7.861964,https://dynamic-media-cdn.tripadvisor.com/medi...,4.0 of 5 bubbles,Historic Sites,60000,35000,1,7.186742
897,898,Goa Cemara Beach,Kabupaten Bantul,Daerah Istimewa Yogyakarta,110.248963,-7.999423,https://dynamic-media-cdn.tripadvisor.com/medi...,4.0 of 5 bubbles,Beaches,110000,85000,2,11.204038
896,897,Tembelan Gorge,Kabupaten Bantul,Daerah Istimewa Yogyakarta,110.429826,-7.940023,https://dynamic-media-cdn.tripadvisor.com/medi...,4.0 of 5 bubbles,Lookouts,110000,85000,1,12.284068
909,910,Kaula Leather Workshop,Kabupaten Bantul,Daerah Istimewa Yogyakarta,110.359355,-7.814600,https://dynamic-media-cdn.tripadvisor.com/medi...,5.0 of 5 bubbles,Lessons & Workshops,90000,65000,2,13.203705
900,901,Jembatan Sawah Sukorame,Kabupaten Bantul,Daerah Istimewa Yogyakarta,110.442600,-7.935847,https://dynamic-media-cdn.tripadvisor.com/medi...,4.5 of 5 bubbles,Bridges,70000,45000,1,13.638092


In [32]:
# Ubah DataFrame menjadi JSON
review_json = item_terrekomendasikan.to_json(orient='records')

# Tampilkan JSON
#print(review_json)


In [33]:
def evaluasimodel(kota_input, durasi_input, items, k=20):
    # Menghitung jumlah item yang direkomendasikan yang relevan (berada di kota yang sama)
    item_terrekomendasikan = recommend_items(kota_input, durasi_input, items, k)
    if item_terrekomendasikan.empty:
        return 0.0, 0.0  # Mengembalikan precision dan recall 0 jika tidak ada item yang ditemukan

    jumlah_item_relevan = len(items[(items['kota'] == kota_input) & (items['attraction_id'].isin(item_terrekomendasikan['attraction_id']))])
    jumlah_item_terrekomendasikan = len(item_terrekomendasikan)

    # Menghitung precision
    precision = jumlah_item_relevan / jumlah_item_terrekomendasikan

    # Menghitung recall
    recall = jumlah_item_relevan / len(items[items['kota'] == kota_input])

    return precision, recall

precision, recall = evaluasimodel(kota_input, durasi_input, items=data[['attraction_id', 'nama', 'kota', 'provinsi','longitude','latitude','img','total_rating','category','adult_price','child_price']], k=20)
print(f'Kota: {kota_input}')
print(f'Durasi: {durasi_input} hari')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')


1/1 [==============================] - 0s 19ms/step
Kota: Kabupaten Bantul
Durasi: 2 hari
Precision: 1.0000
Recall: 0.2500
